# Study Area Analysis & Visualization

## 1. Setup (Run this first)

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D # For custom legends
import numpy as np
import os
import glob

# For map projections and features
import cartopy.crs as ccrs
import cartopy.feature as cfeature

print("--- 1. COMMON DATA LOADING AND PREPARATION (US & Canada) - CORRECTED ID COLS ---")

# --- Configuration ---
BASE_DIR = '../metadata/'
ATTRIBUTES_PATH = os.path.join(BASE_DIR, 'attributes.csv')
CSV_FILTERED_DIR = os.path.join(BASE_DIR, 'csv_filtered')

# CORRECTED Shapefile ID Column Names
US_SHAPEFILE_PATH = os.path.join('shapefiles', 'GL_GAGE2_all.shp')
US_SHP_GAUGE_ID_COL = 'GAGE_ID' # As per your clarification for US stations

CA_SHAPEFILE_PATH = os.path.join('shapefiles', 'ADP02_Basin_Select.shp')
CA_SHP_GAUGE_ID_COL = 'StationNum' # VERIFY this is correct for Canadian. If it's 'StationName', change it.

COLS_TO_LOAD_ATTR = [
    'gauge_id', 'area', 'pre_mm_syr', 'tmp_dc_syr', 'dis_m3_pyr',
    'for_pc_sse', 'crp_pc_sse', 'urb_pc_sse', 'wet_pc_sg1', 'lka_pc_sse',
]

# --- Load and Prepare Data ---
basin_csv_files = glob.glob(os.path.join(CSV_FILTERED_DIR, '*.csv'))
studied_basin_ids = [os.path.splitext(os.path.basename(f))[0] for f in basin_csv_files]
print(f"Found {len(studied_basin_ids)} studied basin IDs from CSV files.")

try:
    attributes_df = pd.read_csv(ATTRIBUTES_PATH, usecols=COLS_TO_LOAD_ATTR)
    attributes_df['gauge_id'] = attributes_df['gauge_id'].astype(str).str.strip()
    print(f"Attributes CSV loaded: {len(attributes_df)} rows.")
except ValueError as e:
    print(f"Error loading attributes CSV: {e}")
    exit()

attributes_study_df = attributes_df[attributes_df['gauge_id'].isin(studied_basin_ids)].copy()
print(f"Attributes data filtered to {len(attributes_study_df)} studied basins.")
if attributes_study_df.empty:
    print("No matching basins found between attributes.csv and filtered CSV files. Exiting.")
    exit()

# --- Load US Shapefile ---
gdf_us = None
try:
    gdf_us_raw = gpd.read_file(US_SHAPEFILE_PATH)
    print(f"US Shapefile '{US_SHAPEFILE_PATH}' loaded with {len(gdf_us_raw)} features. CRS: {gdf_us_raw.crs}")
    if US_SHP_GAUGE_ID_COL not in gdf_us_raw.columns:
        print(f"ERROR: Column '{US_SHP_GAUGE_ID_COL}' not found in US shapefile. Available: {gdf_us_raw.columns.tolist()}")
    else:
        gdf_us = gdf_us_raw[[US_SHP_GAUGE_ID_COL, 'geometry']].copy()
        gdf_us = gdf_us.rename(columns={US_SHP_GAUGE_ID_COL: 'gauge_id_shp'}) # Standardize to 'gauge_id_shp'
        gdf_us['gauge_id_shp'] = gdf_us['gauge_id_shp'].astype(str).str.strip()
        print(f"US GDF processed. Length: {len(gdf_us)}. Target ID col: 'gauge_id_shp'")
except Exception as e:
    print(f"Error loading or processing US Shapefile: {e}")

# --- Load Canadian Shapefile ---
gdf_ca = None
try:
    gdf_ca_raw = gpd.read_file(CA_SHAPEFILE_PATH)
    print(f"Canadian Shapefile '{CA_SHAPEFILE_PATH}' loaded with {len(gdf_ca_raw)} features. CRS: {gdf_ca_raw.crs}")
    if CA_SHP_GAUGE_ID_COL not in gdf_ca_raw.columns:
        print(f"ERROR: Column '{CA_SHP_GAUGE_ID_COL}' not found in Canadian shapefile. Available: {gdf_ca_raw.columns.tolist()}")
    else:
        gdf_ca = gdf_ca_raw[[CA_SHP_GAUGE_ID_COL, 'geometry']].copy()
        gdf_ca = gdf_ca.rename(columns={CA_SHP_GAUGE_ID_COL: 'gauge_id_shp'}) # Standardize to 'gauge_id_shp'
        gdf_ca['gauge_id_shp'] = gdf_ca['gauge_id_shp'].astype(str).str.strip()
        print(f"Canadian GDF processed. Length: {len(gdf_ca)}. Target ID col: 'gauge_id_shp'")
except Exception as e:
    print(f"Error loading or processing Canadian Shapefile: {e}")

# --- Concatenate US and Canadian GeoDataFrames ---
# (Rest of the common data preparation block remains the same as the previous "DEBUGGING" version)
gdf_all_countries_list = []
common_crs_for_concat = None

if gdf_us is not None and not gdf_us.empty:
    common_crs_for_concat = gdf_us.crs
    gdf_all_countries_list.append(gdf_us)
    print(f"US GDF added to list for concatenation. CRS: {gdf_us.crs}")

if gdf_ca is not None and not gdf_ca.empty:
    if common_crs_for_concat is None:
        common_crs_for_concat = gdf_ca.crs
    if gdf_ca.crs != common_crs_for_concat:
        print(f"Reprojecting Canadian GDF from {gdf_ca.crs} to {common_crs_for_concat} for concatenation.")
        try:
            gdf_ca = gdf_ca.to_crs(common_crs_for_concat)
            print(f"Canadian GDF reprojected. New CRS: {gdf_ca.crs}")
        except Exception as e:
            print(f"Error reprojecting Canadian GDF: {e}. Skipping Canadian data for concatenation.")
            gdf_ca = None
    if gdf_ca is not None and not gdf_ca.empty:
         gdf_all_countries_list.append(gdf_ca)
         print("Canadian GDF added to list for concatenation.")

if not gdf_all_countries_list:
    print("ERROR: No shapefile data available for concatenation. Exiting.")
    exit()

gdf_all_countries = gpd.GeoDataFrame(pd.concat(gdf_all_countries_list, ignore_index=True), crs=common_crs_for_concat)
print(f"Combined shapefiles. Total {len(gdf_all_countries)} features. Initial Combined CRS: {gdf_all_countries.crs}")
print(f"Number of unique gauge_id_shp in combined GDF: {gdf_all_countries['gauge_id_shp'].nunique()}")

print(f"Merging gdf_all_countries (gauge_id_shp) with attributes_study_df (gauge_id)")
gdf_study = gdf_all_countries.merge(attributes_study_df, left_on='gauge_id_shp', right_on='gauge_id', how='inner')
print(f"Merged combined shapefile with attributes, resulting in {len(gdf_study)} features for study.")

# Debugging the merge (uncomment if needed):
# common_ids = set(gdf_all_countries['gauge_id_shp'].unique()) & set(attributes_study_df['gauge_id'].unique())
# print(f"Number of common IDs for merge: {len(common_ids)}")
# if len(common_ids) < 10 and len(common_ids) > 0: print(f"Sample common IDs: {list(common_ids)[:min(10, len(common_ids))]}")
# else:
#     print("Sample IDs from gdf_all_countries['gauge_id_shp']:", list(gdf_all_countries['gauge_id_shp'].unique()[:10]))
#     print("Sample IDs from attributes_study_df['gauge_id']:", list(attributes_study_df['gauge_id'].unique()[:10]))


if gdf_study.empty:
    print("No features after merging. This means 'gauge_id_shp' from the combined shapefiles did not match 'gauge_id' in your attributes for the studied basins.")
    exit()

TARGET_CRS = 'EPSG:4326'
if gdf_study.crs != TARGET_CRS:
    print(f"Reprojecting final GDF from {gdf_study.crs} to {TARGET_CRS}")
    gdf_study = gdf_study.to_crs(TARGET_CRS)
else:
    print(f"Final GDF already in target CRS: {TARGET_CRS}")

study_domain_boundary = gdf_study.unary_union
domain_boundary_gdf = gpd.GeoDataFrame(geometry=[study_domain_boundary], crs=gdf_study.crs)

def get_dominant_land_cover(row):
    lc_categories = {
        'Forest': row.get('for_pc_sse', 0), 'Agriculture': row.get('crp_pc_sse', 0),
        'Urban': row.get('urb_pc_sse', 0), 'Wetland': row.get('wet_pc_sg1', 0),
        'Open Water': row.get('lka_pc_sse', 0),
    }
    valid_lc = {k: v for k, v in lc_categories.items() if pd.notna(v)}
    if not valid_lc: return 'Unknown'
    dominant = max(valid_lc, key=valid_lc.get)
    return dominant

required_lc_cols = ['for_pc_sse', 'crp_pc_sse', 'urb_pc_sse', 'wet_pc_sg1', 'lka_pc_sse']
if all(col in gdf_study.columns for col in required_lc_cols):
    gdf_study['dominant_lc'] = gdf_study.apply(get_dominant_land_cover, axis=1)
    lc_colors = {
        'Forest': 'darkgreen', 'Agriculture': 'gold', 'Urban': 'gray',
        'Wetland': 'mediumblue', 'Open Water': 'lightblue', 'Unknown': 'white'
    }
    print("Dominant land cover calculated. Counts:\n", gdf_study['dominant_lc'].value_counts())
else:
    missing_cols_lc = [col for col in required_lc_cols if col not in gdf_study.columns]
    print(f"Skipping dominant land cover (missing columns: {missing_cols_lc}).")
    gdf_study['dominant_lc'] = 'Not Calculated'
    lc_colors = {'Not Calculated': 'white'}

great_lakes_labels = {
    'Lake Superior': (-88.0, 47.5), 'Lake Michigan': (-87.0, 44.0),
    'Lake Huron': (-82.5, 44.5), 'Lake Erie': (-81.0, 42.2),
    'Lake Ontario': (-77.5, 43.7),
}

map_extent = [-94, -73, 39.5, 50]
print(f"Using MANUALLY set map extent: {map_extent}")

def setup_great_lakes_map(ax_map, extent):
    ax_map.set_extent(extent, crs=ccrs.PlateCarree())
    ax_map.add_feature(cfeature.LAND.with_scale('50m'), facecolor='#E0E0E0', edgecolor='gray', zorder=0)
    ax_map.add_feature(cfeature.OCEAN.with_scale('50m'), facecolor='aliceblue', zorder=0)
    ax_map.add_feature(cfeature.LAKES.with_scale('10m'), facecolor='aliceblue', edgecolor='darkgray', zorder=1)
    ax_map.add_feature(cfeature.BORDERS.with_scale('50m'), linestyle=':', edgecolor='black', linewidth=0.7, zorder=2)
    ax_map.add_feature(cfeature.STATES.with_scale('50m'), linestyle=':', edgecolor='dimgray', linewidth=0.5, zorder=2)
    for name, (lon, lat) in great_lakes_labels.items():
        ax_map.text(lon, lat, name, transform=ccrs.Geodetic(),
                    ha='center', va='center', fontsize=7, fontweight='bold',
                    bbox=dict(facecolor='white', alpha=0.5, pad=0.1, edgecolor='none'), zorder=5)
    return ax_map

print("--- COMMON DATA PREPARATION COMPLETE (US & Canada) - CORRECTED ID COLS ---")

--- 1. COMMON DATA LOADING AND PREPARATION (US & Canada) - CORRECTED ID COLS ---
Found 976 studied basin IDs from CSV files.
Attributes CSV loaded: 1088 rows.
Attributes data filtered to 976 studied basins.
US Shapefile 'shapefiles\GL_GAGE2_all.shp' loaded with 439 features. CRS: PROJCS["NAD_1983_Albers",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
US GDF processed. Length: 439. Target ID col: 'gauge_id_shp'
Canadian Shapefile 'shapefiles\ADP02_Basin_Select.shp' loaded with 649 features. CRS: PROJCS["Canada_A

C:\Users\ybrot\AppData\Local\Temp\ipykernel_7364\26428224.py:139: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  study_domain_boundary = gdf_study.unary_union


Dominant land cover calculated. Counts:
 dominant_lc
Agriculture    361
Forest         339
Open Water     208
Urban           56
Wetland         12
Name: count, dtype: int64
Using MANUALLY set map extent: [-94, -73, 39.5, 50]
--- COMMON DATA PREPARATION COMPLETE (US & Canada) - CORRECTED ID COLS ---


In [2]:
gdf_study.columns

Index(['gauge_id_shp', 'geometry', 'gauge_id', 'crp_pc_sse', 'wet_pc_sg1',
       'for_pc_sse', 'pre_mm_syr', 'tmp_dc_syr', 'urb_pc_sse', 'lka_pc_sse',
       'dis_m3_pyr', 'area', 'dominant_lc'],
      dtype='object')

## 2. Figure 1a (This version is not used)

In [4]:
print("\n--- GENERATING FIGURE 1: Study Domain and Land Cover (US & Canada) ---")

# Ensure the common block has been run and gdf_study, domain_boundary_gdf are available
if 'gdf_study' not in globals() or 'domain_boundary_gdf' not in globals() or gdf_study.empty: # Added gdf_study.empty check
    print("ERROR: Common data preparation block not run or gdf_study is empty. Please run it first.")
else:
    fig1, (ax1a, ax1b) = plt.subplots(1, 2, figsize=(16, 8), # Width, Height
                                   subplot_kw={'projection': ccrs.LambertConformal(central_longitude=-85, central_latitude=45)},
                                   constrained_layout=True) # Use constrained_layout

    # Panel A: Geographic Location of Study Basins
    ax1a = setup_great_lakes_map(ax1a, map_extent)
    ax1a.set_title('A) Geographic Location of Studied Basins', fontsize=11)
    gdf_study.plot(ax=ax1a, transform=ccrs.PlateCarree(),
                   facecolor='skyblue', edgecolor='darkblue', linewidth=0.3, alpha=0.6, zorder=3)
    if not domain_boundary_gdf.empty: # Check if domain_boundary_gdf is not empty
        domain_boundary_gdf.plot(ax=ax1a, transform=ccrs.PlateCarree(),
                                 facecolor='none', edgecolor='red', linewidth=1.2, linestyle='--', zorder=4)
    handles_a = [
        Line2D([0], [0], marker='s', color='w', label='Studied Basins Extent', markerfacecolor='skyblue', markeredgecolor='darkblue', markersize=8),
        Line2D([0], [0], color='red', lw=1.2, linestyle='--', label='Overall Study Domain Boundary')
    ]
    # Place legend consistently
    ax1a.legend(handles=handles_a, loc='lower left', fontsize=8, title_fontsize=9, frameon=False) # Added frameon=False for cleaner look
    ax1a.text(0.01, 0.01, '(Basemap: Natural Earth)', transform=ax1a.transAxes, fontsize=6, ha='left', va='bottom', color='dimgray')


    # Panel B: Generalized Land Cover of Studied Basins
    ax1b = setup_great_lakes_map(ax1b, map_extent)
    ax1b.set_title('B) Dominant Land Cover of Studied Basins', fontsize=11)
    legend_elements_lc = []
    # Ensure consistent order in legend for land cover categories
    # Use lc_colors keys for a predefined order if that's desirable,
    # otherwise gdf_study['dominant_lc'].unique() is fine but order might vary run-to-run.
    # Let's use the order from lc_colors if all are present, otherwise unique values.
    
    plot_lc_types = []
    if 'dominant_lc' in gdf_study.columns and gdf_study['dominant_lc'].nunique() > 1 and gdf_study['dominant_lc'].iloc[0] != 'Not Calculated':
        # Prefer order from lc_colors if defined
        defined_lc_order = [lc for lc in lc_colors.keys() if lc in gdf_study['dominant_lc'].unique() and lc not in ['Not Calculated', 'Unknown']]
        if not defined_lc_order: # Fallback if lc_colors doesn't match unique values well
            defined_lc_order = sorted([lc for lc in gdf_study['dominant_lc'].unique() if lc not in ['Not Calculated', 'Unknown']])

        for lc_type in defined_lc_order:
            color = lc_colors.get(lc_type, 'purple') # Default color if somehow missing from lc_colors
            gdf_subset = gdf_study[gdf_study['dominant_lc'] == lc_type]
            if not gdf_subset.empty:
                gdf_subset.plot(ax=ax1b, transform=ccrs.PlateCarree(),
                                facecolor=color, edgecolor='black', linewidth=0.1, label=lc_type, zorder=3)
                legend_elements_lc.append(Line2D([0], [0], marker='s', color='w', label=lc_type,
                                              markerfacecolor=color, markersize=8))
        if legend_elements_lc:
            # Place legend consistently
            ax1b.legend(handles=legend_elements_lc, title="Dominant Land Cover", loc='lower left', fontsize=8, title_fontsize=9, frameon=False)
    else:
        gdf_study.plot(ax=ax1b, transform=ccrs.PlateCarree(), facecolor='lightgrey', edgecolor='black', linewidth=0.1, zorder=3)
        ax1b.text(0.5, 0.5, "Dominant Land Cover Not Plotted\n(Data missing or uniform)",
                  transform=ax1b.transAxes, ha='center', va='center', color='red', fontsize=9)
    ax1b.text(0.01, 0.01, '(Basemap: Natural Earth)', transform=ax1b.transAxes, fontsize=6, ha='left', va='bottom', color='dimgray')

    # fig1.suptitle("Figure 1: Study Domain and Dominant Land Cover", fontsize=14) # Optional overall title

    # No plt.tight_layout() needed if constrained_layout=True is used effectively.
    # If constrained_layout=True is not giving desired results, remove it and uncomment plt.tight_layout(pad=...)
    # plt.tight_layout(pad=2.0) # Try this if constrained_layout is not satisfactory

    plt.savefig('Figure1_Study_Domain_Land_Cover_Combined.png', dpi=300, bbox_inches='tight')
    print("Figure 1 saved as Figure1_Study_Domain_Land_Cover_Combined.png")
    plt.close(fig1)


--- GENERATING FIGURE 1: Study Domain and Land Cover (US & Canada) ---
Figure 1 saved as Figure1_Study_Domain_Land_Cover_Combined.png


## 3. Figure 1b

In [5]:
import pandas as pd # Ensure pandas is imported if not already from common block
import numpy as np # Ensure numpy is imported if not already from common block
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
# Assuming setup_great_lakes_map, map_extent, and gdf_study are available from the common block

print("\n--- GENERATING FIGURE 2: Climatic Gradients (with F to C conversion) ---")

if 'gdf_study' not in globals() or gdf_study.empty:
    print("ERROR: Common data preparation block not run or gdf_study is empty. Please run it first.")
else:
    fig2, (ax2a, ax2b) = plt.subplots(1, 2, figsize=(16, 8),
                                   subplot_kw={'projection': ccrs.LambertConformal(central_longitude=-85, central_latitude=45)})

    # Create a copy for modifications to avoid SettingWithCopyWarning if gdf_study is a slice
    gdf_plot_fig2 = gdf_study.copy()

    # Panel A: Mean Annual Precipitation
    ax2a = setup_great_lakes_map(ax2a, map_extent) # setup_great_lakes_map should be defined in your common block
    ax2a.set_title('A) Mean Annual Precipitation (mm/year)', fontsize=11)
    if 'pre_mm_syr' in gdf_plot_fig2.columns:
        gdf_plot_fig2.plot(ax=ax2a, transform=ccrs.PlateCarree(), column='pre_mm_syr',
                           cmap='Blues', legend=True,
                           legend_kwds={'label': "Precipitation (mm/year)", 'orientation': "horizontal", 'shrink': 0.5, 'pad': 0.1, 'aspect': 30},
                           edgecolor='darkgray', linewidth=0.1, missing_kwds={'color': 'lightgrey', "hatch": "///", "label": "Missing values"})
    else:
        ax2a.text(0.5, 0.5, "Precipitation Data ('pre_mm_syr') Not Found", transform=ax2a.transAxes, ha='center', va='center', color='red', fontsize=9)
    ax2a.text(0.01, 0.01, '(Basemap: Natural Earth)', transform=ax2a.transAxes, fontsize=6, ha='left', va='bottom', color='dimgray')

    # Panel B: Mean Annual Air Temperature
    ax2b = setup_great_lakes_map(ax2b, map_extent)
    ax2b.set_title('B) Mean Annual Air Temperature (°C)', fontsize=11)
    if 'tmp_dc_syr' in gdf_plot_fig2.columns:
        # Convert Fahrenheit to Celsius
        # Ensure the column is numeric before conversion
        if pd.api.types.is_numeric_dtype(gdf_plot_fig2['tmp_dc_syr']):
            gdf_plot_fig2['tmp_dc_syr_C'] = (gdf_plot_fig2['tmp_dc_syr'] - 32) * 5/9
            print("Temperature column 'tmp_dc_syr' converted from F to C as 'tmp_dc_syr_C'.")

            gdf_plot_fig2.plot(ax=ax2b, transform=ccrs.PlateCarree(), column='tmp_dc_syr_C', # Plot the new Celsius column
                               cmap='coolwarm', legend=True,
                               legend_kwds={'label': "Temperature (°C)", 'orientation': "horizontal", 'shrink': 0.5, 'pad': 0.1, 'aspect': 30},
                               edgecolor='darkgray', linewidth=0.1, missing_kwds={'color': 'lightgrey', "hatch": "///", "label": "Missing values"})
        else:
            print("ERROR: Temperature column 'tmp_dc_syr' is not numeric. Cannot convert F to C.")
            ax2b.text(0.5, 0.5, "Temperature Data ('tmp_dc_syr')\nNot Numeric for Conversion", transform=ax2b.transAxes, ha='center', va='center', color='red', fontsize=9)
    else:
        ax2b.text(0.5, 0.5, "Temperature Data ('tmp_dc_syr') Not Found", transform=ax2b.transAxes, ha='center', va='center', color='red', fontsize=9)
    ax2b.text(0.01, 0.01, '(Basemap: Natural Earth)', transform=ax2b.transAxes, fontsize=6, ha='left', va='bottom', color='dimgray')

    plt.tight_layout(pad=1.5)
    plt.savefig('Figure2_Climatic_Gradients_Celsius.png', dpi=300, bbox_inches='tight')
    print("Figure 2 saved as Figure2_Climatic_Gradients_Celsius.png")
    plt.close(fig2)


--- GENERATING FIGURE 2: Climatic Gradients (with F to C conversion) ---
Temperature column 'tmp_dc_syr' converted from F to C as 'tmp_dc_syr_C'.
Figure 2 saved as Figure2_Climatic_Gradients_Celsius.png


## 4. Figure 1c

In [15]:
# This code should run AFTER your "Common Data Loading and Preparation Block"
# AND AFTER the "Phase 1: Compute Numerical Values" block (or at least the part
# where 'mean_discharge_m3s_manual' is calculated and added to gdf_study).

print("\n--- GENERATING FIGURE 3: Distribution of Key Basin Characteristics (Using Manually Computed Discharge) ---")

if 'gdf_study' not in globals() or gdf_study.empty:
    print("ERROR: Common data preparation block not run or gdf_study is empty. Please run it first.")
elif 'mean_discharge_m3s_manual' not in gdf_study.columns:
    print("ERROR: 'mean_discharge_m3s_manual' column not found in gdf_study. Ensure manual calculation was run.")
else:
    fig3, (ax3a, ax3b) = plt.subplots(1, 2, figsize=(10, 4)) # Smaller figure

    # Panel A: Drainage Areas (remains the same, uses 'area' column)
    if 'area' in gdf_study.columns:
        areas_km2 = gdf_study['area'].dropna()
        if not areas_km2.empty:
            # Determine if log scale is appropriate based on range/skewness
            if (areas_km2.max() / areas_km2.min() > 100) and areas_km2.min() > 0 : # Heuristic for log scale
                ax3a.hist(np.log10(areas_km2), bins=25, color='skyblue', edgecolor='black')
                ax3a.set_xlabel('Log₁₀(Drainage Area [km²])')
            else:
                ax3a.hist(areas_km2, bins=25, color='skyblue', edgecolor='black')
                ax3a.set_xlabel('Drainage Area (km²)')
        else:
            ax3a.text(0.5, 0.5, "Area Data Missing\nor All NaN", transform=ax3a.transAxes, ha='center', va='center', color='red')
    else:
        ax3a.text(0.5, 0.5, "Area Column Missing", transform=ax3a.transAxes, ha='center', va='center', color='red')
    ax3a.set_ylabel('Number of Basins')
    ax3a.set_title('A) Distribution of Drainage Areas', fontsize=10)
    ax3a.grid(axis='y', linestyle='--', alpha=0.7)
    ax3a.tick_params(axis='both', which='major', labelsize=8)


    # Panel B: Mean Annual Discharge (using the new 'mean_discharge_m3s_manual' column)
    discharge_column_for_plot = 'mean_discharge_m3s_manual'
    if discharge_column_for_plot in gdf_study.columns:
        # Use the manually calculated mean discharge for each basin
        discharge_values_for_plot = gdf_study[discharge_column_for_plot].dropna()

        if not discharge_values_for_plot.empty:
            print(f"\nPlotting histogram for '{discharge_column_for_plot}':")
            print(discharge_values_for_plot.describe()) # See stats of what's being plotted

            # Determine if log scale is appropriate
            min_q_plot = discharge_values_for_plot.min()
            max_q_plot = discharge_values_for_plot.max()

            if min_q_plot > 0 and (max_q_plot / min_q_plot > 100): # Heuristic for log scale
                ax3b.hist(np.log10(discharge_values_for_plot), bins=25, color='lightcoral', edgecolor='black')
                ax3b.set_xlabel('Log₁₀(Mean Annual Discharge [m³/s])')
            else:
                ax3b.hist(discharge_values_for_plot, bins=25, color='lightcoral', edgecolor='black')
                ax3b.set_xlabel('Mean Annual Discharge (m³/s)')
        else:
            ax3b.text(0.5, 0.5, "Manually Calculated\nDischarge Data Missing\nor All NaN", transform=ax3b.transAxes, ha='center', va='center', color='red', fontsize=8)
    else:
        ax3b.text(0.5, 0.5, f"Column '{discharge_column_for_plot}'\nMissing from gdf_study", transform=ax3b.transAxes, ha='center', va='center', color='red', fontsize=8)

    ax3b.set_ylabel('Number of Basins')
    ax3b.set_title('B) Distribution of Mean Annual Discharge', fontsize=10)
    ax3b.grid(axis='y', linestyle='--', alpha=0.7)
    ax3b.tick_params(axis='both', which='major', labelsize=8)


    plt.tight_layout(pad=1.0)
    plt.savefig('Figure3_Basin_Characteristics_Distribution_ManualQ.png', dpi=300, bbox_inches='tight')
    print("Figure 3 saved as Figure3_Basin_Characteristics_Distribution_ManualQ.png")
    plt.close(fig3)

print("\n--- FIGURE 3 GENERATION ATTEMPTED (Using Manually Computed Discharge) ---")


--- GENERATING FIGURE 3: Distribution of Key Basin Characteristics (Using Manually Computed Discharge) ---

Plotting histogram for 'mean_discharge_m3s_manual':
count    858.000000
mean      14.153692
std       27.919092
min        0.038030
25%        1.177993
50%        3.595899
75%       12.208385
max      204.097644
Name: mean_discharge_m3s_manual, dtype: float64
Figure 3 saved as Figure3_Basin_Characteristics_Distribution_ManualQ.png

--- FIGURE 3 GENERATION ATTEMPTED (Using Manually Computed Discharge) ---


In [17]:
import pandas as pd
import numpy as np
import os
import glob
import json # For JSON output

# This code should run AFTER your "Common Data Loading and Preparation Block"
# where gdf_study is created.

print("\n--- MANUALLY CALCULATING MEAN ANNUAL DISCHARGE & COMPUTING STATS ---")

# Dictionary to store all computed stats for JSON output
output_stats = {}

if 'gdf_study' not in globals() or gdf_study.empty:
    print("ERROR: gdf_study not found or is empty. Run the common data preparation block first.")
    # Initialize with NaNs or empty structures if gdf_study is missing
    output_stats['error'] = "gdf_study not found or empty"
    num_studied_basins = 0
    min_drainage_area_km2, max_drainage_area_km2, median_drainage_area_km2 = [np.nan]*3
    min_discharge_m3s, max_discharge_m3s, median_discharge_m3s, mean_discharge_m3s = [np.nan]*4
    min_precip_mm_yr, max_precip_mm_yr, median_precip_mm_yr, mean_precip_mm_yr = [np.nan]*4
    min_temp_c, max_temp_c, median_temp_c, mean_temp_c = [np.nan]*4
    dominant_lc_counts_dict = {}
else:
    mean_discharges_m3s_manual = {}
    basins_with_no_valid_q_data = [] # List to collect basins with issues

    print("Calculating mean discharge from daily CSVs...")
    for gauge_id_str in gdf_study['gauge_id'].unique():
        daily_csv_path = os.path.join(CSV_FILTERED_DIR, f"{gauge_id_str}.csv")
        if os.path.exists(daily_csv_path):
            try:
                daily_df = pd.read_csv(daily_csv_path)
                if 'discharge' in daily_df.columns and pd.api.types.is_numeric_dtype(daily_df['discharge']):
                    valid_discharges = daily_df['discharge'].dropna()
                    if not valid_discharges.empty:
                        mean_discharges_m3s_manual[gauge_id_str] = valid_discharges.mean()
                    else:
                        basins_with_no_valid_q_data.append(gauge_id_str) # Collect problematic basin ID
                        mean_discharges_m3s_manual[gauge_id_str] = np.nan
                else: # Missing 'discharge' or not numeric
                    basins_with_no_valid_q_data.append(gauge_id_str)
                    mean_discharges_m3s_manual[gauge_id_str] = np.nan
            except Exception: # Catch any error during file processing
                basins_with_no_valid_q_data.append(gauge_id_str)
                mean_discharges_m3s_manual[gauge_id_str] = np.nan
        else: # File not found
            basins_with_no_valid_q_data.append(gauge_id_str)
            mean_discharges_m3s_manual[gauge_id_str] = np.nan

# ... (inside the 'else' block after the loop for manual_discharge_series calculation) ...

    if basins_with_no_valid_q_data:
        print(f"Summary: {len(basins_with_no_valid_q_data)} basins had no valid discharge data or issues during processing.")
        # print(f"Basins with issues (first 10): {basins_with_no_valid_q_data[:10]}")

    # Create the Series with the desired name
    manual_discharge_series = pd.Series(mean_discharges_m3s_manual, name='mean_discharge_m3s_manual')

    # --- CORRECTED MERGE ---
    # Merge the series into gdf_study.
    # The new column in gdf_study will be named after the series' name.
    gdf_study_before_merge_cols = gdf_study.columns.tolist() # For debugging
    gdf_study = gdf_study.merge(manual_discharge_series,
                                left_on='gauge_id',
                                right_index=True,
                                how='left')
    gdf_study_after_merge_cols = gdf_study.columns.tolist() # For debugging

    print(f"Manual mean discharge calculation complete and merged.")
    print(f"Columns before merge: {gdf_study_before_merge_cols}")
    print(f"Columns after merge: {gdf_study_after_merge_cols}") # Check if 'mean_discharge_m3s_manual' is here

    # Verify if the column was added with the correct name
    if 'mean_discharge_m3s_manual' not in gdf_study.columns:
        print("ERROR: 'mean_discharge_m3s_manual' column was NOT successfully added after merge.")
        # Check if it was added with a different name, e.g., 0 or the original Series name if it wasn't set
        # This can happen if 'gauge_id' had issues or if the Series had no name.
        # Our Series DOES have a name, so this shouldn't be the primary issue unless 'gauge_id' had issues.
        # One possible fallback name pandas might use is 0 if the series name was None.
        if 0 in gdf_study.columns and 'mean_discharge_m3s_manual' not in gdf_study_before_merge_cols:
            print("A column named '0' was added. Attempting to rename it to 'mean_discharge_m3s_manual'.")
            gdf_study = gdf_study.rename(columns={0: 'mean_discharge_m3s_manual'})
        elif 'mean_discharge_m3s_manual_x' in gdf_study.columns: # Pandas might add suffixes
             print("Column 'mean_discharge_m3s_manual_x' found, renaming to 'mean_discharge_m3s_manual'.")
             gdf_study = gdf_study.rename(columns={'mean_discharge_m3s_manual_x': 'mean_discharge_m3s_manual'})
        elif 'mean_discharge_m3s_manual_y' in gdf_study.columns:
             print("Column 'mean_discharge_m3s_manual_y' found, renaming to 'mean_discharge_m3s_manual'.")
             gdf_study = gdf_study.rename(columns={'mean_discharge_m3s_manual_y': 'mean_discharge_m3s_manual'})

    if 'mean_discharge_m3s_manual' not in gdf_study.columns:
         print("CRITICAL ERROR: Still unable to find/create 'mean_discharge_m3s_manual' column. Halting statistics.")
         # Initialize stats with NaNs or empty to prevent further errors
         output_stats['error'] = "Failed to create 'mean_discharge_m3s_manual' column."
         min_discharge_m3s, max_discharge_m3s, median_discharge_m3s, mean_discharge_m3s = [np.nan]*4
         # ... initialize other stats as None/NaN ...
    else:
        print("Successfully found/created 'mean_discharge_m3s_manual' column.")
        # --- Now calculate all statistics (the rest of your Phase 1 code) ---
        # This part should now work correctly as 'mean_discharge_m3s_manual' will be the column name.
        output_stats['num_total_basins_in_gdf_study'] = len(gdf_study)
        discharge_column_final = 'mean_discharge_m3s_manual' # This should now match
        num_basins_with_valid_q = gdf_study[discharge_column_final].notna().sum()
        # ... rest of your statistics calculations ...

# ... (The rest of your Phase 1 script for other statistics and JSON output) ...
    output_stats['num_basins_with_valid_q_manual'] = int(num_basins_with_valid_q) # Ensure int for JSON
    print(f"Total number of basins with valid manually calculated discharge: {num_basins_with_valid_q}")


    # 2. Drainage Area Statistics (for basins with valid Q)
    # It might be more consistent to report area stats for the same set of basins for which Q is reported
    gdf_for_stats = gdf_study[gdf_study[discharge_column_final].notna()].copy() # Use only basins with valid Q
    if not gdf_for_stats.empty and 'area' in gdf_for_stats.columns:
        min_drainage_area_km2 = gdf_for_stats['area'].min()
        max_drainage_area_km2 = gdf_for_stats['area'].max()
        median_drainage_area_km2 = gdf_for_stats['area'].median()
        output_stats['drainage_area_km2'] = {
            'min': float(min_drainage_area_km2), 'max': float(max_drainage_area_km2),
            'median': float(median_drainage_area_km2)
        }
        print(f"Drainage Area (km²) for basins with Q: Min={min_drainage_area_km2:.2f}, Max={max_drainage_area_km2:.0f}, Median={median_drainage_area_km2:.1f}")
    else:
        print("WARNING: 'area' column not found or no basins with valid Q for area stats.")
        output_stats['drainage_area_km2'] = {'min': None, 'max': None, 'median': None}


    # 3. Mean Annual Discharge Statistics (from manually calculated daily data)
    if not gdf_for_stats.empty and discharge_column_final in gdf_for_stats.columns:
        valid_manual_discharges = gdf_for_stats[discharge_column_final].dropna() # Should be mostly non-NaN here
        if not valid_manual_discharges.empty:
            min_discharge_m3s = valid_manual_discharges.min()
            max_discharge_m3s = valid_manual_discharges.max()
            median_discharge_m3s = valid_manual_discharges.median()
            mean_discharge_m3s = valid_manual_discharges.mean()
            output_stats['mean_annual_discharge_m3s'] = {
                'min': float(min_discharge_m3s), 'max': float(max_discharge_m3s),
                'median': float(median_discharge_m3s), 'mean': float(mean_discharge_m3s)
            }
            print(f"Mean Annual Discharge (m³/s) from daily data:")
            print(f"  Min:    {min_discharge_m3s:.4g}, Max:    {max_discharge_m3s:.4g}")
            print(f"  Median: {median_discharge_m3s:.4g}, Mean:   {mean_discharge_m3s:.4g}")
        else:
            output_stats['mean_annual_discharge_m3s'] = {'min': None, 'max': None, 'median': None, 'mean': None}
    else:
        print(f"WARNING: Column '{discharge_column_final}' not found or no basins with valid Q for discharge stats.")
        output_stats['mean_annual_discharge_m3s'] = {'min': None, 'max': None, 'median': None, 'mean': None}


    # 4. Mean Annual Precipitation Statistics (for basins with valid Q)
    if not gdf_for_stats.empty and 'pre_mm_syr' in gdf_for_stats.columns:
        min_precip_mm_yr = gdf_for_stats['pre_mm_syr'].min()
        max_precip_mm_yr = gdf_for_stats['pre_mm_syr'].max()
        median_precip_mm_yr = gdf_for_stats['pre_mm_syr'].median()
        mean_precip_mm_yr = gdf_for_stats['pre_mm_syr'].mean()
        output_stats['mean_annual_precipitation_mm_yr'] = {
            'min': float(min_precip_mm_yr), 'max': float(max_precip_mm_yr),
            'median': float(median_precip_mm_yr), 'mean': float(mean_precip_mm_yr)
        }
        print(f"Mean Annual Precipitation (mm/year) for basins with Q: Min={min_precip_mm_yr:.0f}, Max={max_precip_mm_yr:.0f}, Median={median_precip_mm_yr:.0f}, Mean={mean_precip_mm_yr:.0f}")
    else:
        print("WARNING: 'pre_mm_syr' column not found or no basins with valid Q for precip stats.")
        output_stats['mean_annual_precipitation_mm_yr'] = {'min': None, 'max': None, 'median': None, 'mean': None}


    # 5. Mean Annual Temperature Statistics (for basins with valid Q)
    if not gdf_for_stats.empty and 'tmp_dc_syr' in gdf_for_stats.columns and pd.api.types.is_numeric_dtype(gdf_for_stats['tmp_dc_syr']):
        # Ensure Celsius column exists on gdf_for_stats
        if 'tmp_dc_syr_C' not in gdf_for_stats.columns:
             gdf_for_stats['tmp_dc_syr_C'] = (gdf_for_stats['tmp_dc_syr'] - 32) * 5/9

        if 'tmp_dc_syr_C' in gdf_for_stats.columns and gdf_for_stats['tmp_dc_syr_C'].notna().any():
            min_temp_c = gdf_for_stats['tmp_dc_syr_C'].min()
            max_temp_c = gdf_for_stats['tmp_dc_syr_C'].max()
            median_temp_c = gdf_for_stats['tmp_dc_syr_C'].median()
            mean_temp_c = gdf_for_stats['tmp_dc_syr_C'].mean()
            output_stats['mean_annual_temperature_c'] = {
                'min': float(min_temp_c), 'max': float(max_temp_c),
                'median': float(median_temp_c), 'mean': float(mean_temp_c)
            }
            print(f"Mean Annual Temperature (°C) for basins with Q: Min={min_temp_c:.1f}, Max={max_temp_c:.1f}, Median={median_temp_c:.1f}, Mean={mean_temp_c:.1f}")
        else:
            output_stats['mean_annual_temperature_c'] = {'min': None, 'max': None, 'median': None, 'mean': None}
    else:
        print("WARNING: Temp data ('tmp_dc_syr') not found/numeric or no basins with valid Q for temp stats.")
        output_stats['mean_annual_temperature_c'] = {'min': None, 'max': None, 'median': None, 'mean': None}


    # 6. Land Cover Distribution (for basins with valid Q)
    if not gdf_for_stats.empty and 'dominant_lc' in gdf_for_stats.columns and gdf_for_stats['dominant_lc'].iloc[0] != 'Not Calculated':
        dominant_lc_counts = gdf_for_stats['dominant_lc'].value_counts(normalize=True) * 100
        dominant_lc_counts_dict = {k: float(v) for k, v in dominant_lc_counts.items()} # Convert to dict of floats
        output_stats['dominant_land_cover_percentage'] = dominant_lc_counts_dict
        print("\nPercentage of basins (with valid Q) by dominant land cover type:")
        for lc_type, percentage in dominant_lc_counts.items():
            print(f"- {lc_type}: {percentage:.0f}%")
    else:
        print("WARNING: Dominant land cover not calculated or no basins with valid Q for LC stats.")
        output_stats['dominant_land_cover_percentage'] = {}

# Save to JSON
json_output_path = 'study_domain_statistics.json'
try:
    with open(json_output_path, 'w') as f:
        json.dump(output_stats, f, indent=4, allow_nan=True) # allow_nan is important
    print(f"\nNumerical statistics saved to: {json_output_path}")
except Exception as e:
    print(f"Error saving statistics to JSON: {e}")


print("\n--- NUMERICAL VALUE COMPUTATION AND JSON EXPORT COMPLETE ---")


--- MANUALLY CALCULATING MEAN ANNUAL DISCHARGE & COMPUTING STATS ---
Calculating mean discharge from daily CSVs...
Summary: 118 basins had no valid discharge data or issues during processing.
Manual mean discharge calculation complete and merged.
Columns before merge: ['gauge_id_shp', 'geometry', 'gauge_id', 'crp_pc_sse', 'wet_pc_sg1', 'for_pc_sse', 'pre_mm_syr', 'tmp_dc_syr', 'urb_pc_sse', 'lka_pc_sse', 'dis_m3_pyr', 'area', 'dominant_lc', 'dis_m3_s', 'tmp_dc_syr_C', 'mean_discharge_m3s_manual_x', 'mean_discharge_m3s_manual_y']
Columns after merge: ['gauge_id_shp', 'geometry', 'gauge_id', 'crp_pc_sse', 'wet_pc_sg1', 'for_pc_sse', 'pre_mm_syr', 'tmp_dc_syr', 'urb_pc_sse', 'lka_pc_sse', 'dis_m3_pyr', 'area', 'dominant_lc', 'dis_m3_s', 'tmp_dc_syr_C', 'mean_discharge_m3s_manual_x', 'mean_discharge_m3s_manual_y', 'mean_discharge_m3s_manual']
Successfully found/created 'mean_discharge_m3s_manual' column.
Total number of basins with valid manually calculated discharge: 858
Drainage Area (k